In [1]:
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import gradient_descent_v2
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

def eachFile(filepath):
	pathDir = os.listdir(filepath)
	out = []
	for allDir in pathDir:
		child = allDir
		out.append(child)
	return out


NUM_CLASSES = 10
TRAIN_PATH = 'dataset/train/'
TEST_PATH = 'dataset/test/'

FC_NUMS = 4096

FREEZE_LAYERS = 17

IMAGE_SIZE = 224

base_model = VGG19(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(4096, activation='relu')(x)
prediction = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=prediction)

for layer in model.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in model.layers[FREEZE_LAYERS:]:
    layer.trainable = True

model.compile(optimizer=gradient_descent_v2.SGD(learning_rate=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(directory=TRAIN_PATH,
                                                    target_size=(IMAGE_SIZE, IMAGE_SIZE), classes=eachFile(TRAIN_PATH))
filepath = 'model/sewer_weight.h5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

history_ft = model.fit_generator(train_generator, epochs=10, callbacks=[checkpoint])
model.save('model/sewer_weight_final.h5')

Init Plugin
Init Graph Optimizer
Init Kernel
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-05-06 20:56:07.158454: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-06 20:56:07.158565: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 600 images belonging to 10 classes.


/Users/iyunji/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-05-06 20:56:08.150558: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-06 20:56:08.150780: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10


2022-05-06 20:56:15.198461: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 19s 498ms/step - loss: 2.1129 - accuracy: 0.4218

Epoch 00001: loss improved from inf to 1.54548, saving model to model/sewer_weight.h5
Epoch 2/10
19/19 [==============================] - 9s 474ms/step - loss: 0.3092 - accuracy: 0.9048

Epoch 00002: loss improved from 1.54548 to 0.30482, saving model to model/sewer_weight.h5
Epoch 3/10
19/19 [==============================] - 9s 475ms/step - loss: 0.1411 - accuracy: 0.9600

Epoch 00003: loss improved from 0.30482 to 0.11571, saving model to model/sewer_weight.h5
Epoch 4/10
19/19 [==============================] - 9s 476ms/step - loss: 0.0750 - accuracy: 0.9781

Epoch 00004: loss improved from 0.11571 to 0.09564, saving model to model/sewer_weight.h5
Epoch 5/10
19/19 [==============================] - 10s 482ms/step - loss: 0.0354 - accuracy: 0.9917

Epoch 00005: loss improved from 0.09564 to 0.03257, saving model to model/sewer_weight.h5
Epoch 6/10
19/19 [==============================] - 9s 472

In [1]:

   
from tensorflow.python.keras.initializers import glorot_uniform
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import cv2
from keras.utils import CustomObjectScope
from keras.preprocessing.image import  img_to_array

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

# image folder
folder_path = 'dataset/test/'

# path to model
model_path = 'model/sewer_weight_9.h5'
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model = load_model(model_path)

# dimensions of images
img_width, img_height = 224, 224


i = 0
images = []
for img in os.listdir(folder_path):
   img1 = image.load_img(os.path.join(folder_path, img), target_size=(img_width, img_height))
   img2 = img_to_array(img1)
   img2 = np.expand_dims(img2, axis=0)
   classes = model.predict(img2)[0]
   idxs = np.argsort(classes)[::-1][:1]

   classname = ['IN', 'JOINT,FAULTY', 'PIPE,BROKEN', 'OUT', 'DEBRIS,SILTY', 'JOINT,OPEN',
                'HORIZONTAL,CRACK', 'VERTICAL,CRACK', 'LATERAL,PROTRUDING']

   out = cv2.imread(os.path.join(folder_path, img))

   for (i, j) in enumerate(idxs):
       label = "{}:{:.2f}%".format(classname[idxs[i]], classes[idxs[i]] * 100)
       cv2.putText(out, label, (10, (i * 30) + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

   cv2.imwrite("visualization/sd/%s"%img,out)

Init Plugin
Init Graph Optimizer
Init Kernel


AttributeError: module 'tensorflow.compat.v2.__internal__' has no attribute 'register_clear_session_function'